In [119]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from tqdm import tqdm


In [120]:
df = pd.read_csv('data/train.csv')
# df_test = pd.read_csv('data/test.csv')

print(df.shape)
# print(df_test.shape)

(4219370, 21)


In [121]:
# filter incorrect
df = df[(df['distance'] > 0) & (df['y'] > 0)]
df['avg'] = df['distance'] / df['y']
df = df[df['avg'] <= 100]


In [122]:
df = df[['st_code_snd', 'st_code_rsv', 'distance', 'y']]

In [123]:
df['predict'] = None

In [124]:
def join_row(row):
    return f"{row['st_code_snd']} {row['st_code_rsv']} {row['distance']}"

df['combo'] = df.apply(join_row, axis=1)

In [125]:
df.head()

,st_code_snd,st_code_rsv,distance,y,predict,combo
0,wagonwagonwagontutu:)wagon,wagondigitalnewyearrailway:)happy,930.0,185.200000,None,wagonwagonwagontutu:)wagon wagondigitalnewyear...
1,railwaypgkhappyrailway:)railway,digitalhappynewyeardigital:)wagon,16.0,0.250000,None,railwaypgkhappyrailway:)railway digitalhappyne...
2,railwaytutupgkpgk:)digital,pgkdigitalrailway:):)happy,879.0,109.133333,None,railwaytutupgkpgk:)digital pgkdigitalrailway:)...
3,tuturailwaydigital2022:)digital,newyearnewyear20222022:)2022,8569.0,214.093056,None,tuturailwaydigital2022:)digital newyearnewyear...
4,railwaytutupgkpgk:)digital,pgkhappyrailwayhack:)railway,656.0,104.100000,None,railwaytutupgkpgk:)digital pgkhappyrailwayhack...


In [126]:
l = df['combo'].unique().tolist()

In [127]:
l

['wagonwagonwagontutu:)wagon wagondigitalnewyearrailway:)happy 930.0',
 'railwaypgkhappyrailway:)railway digitalhappynewyeardigital:)wagon 16.0',
 'railwaytutupgkpgk:)digital pgkdigitalrailway:):)happy 879.0',
 'tuturailwaydigital2022:)digital newyearnewyear20222022:)2022 8569.0',
 'railwaytutupgkpgk:)digital pgkhappyrailwayhack:)railway 656.0',
 'hacknewyearrailway:)pgk hackwagonhappy:)newyear 1021.0',
 'happynewyearpgk:)railway hackwagon:):)tutu 4480.0',
 'tutuwagon2022hack:)pgk newyearwagondigitalnewyear:)tutu 209.0',
 'railwaytutupgkhappy:)newyear digitalhappynewyeardigital:)wagon 1680.0',
 'newyearrailwaypgknewyear:)hack 2022wagondigital:)digital 3832.0',
 'happy:)pgkrailway:)2022 wagonhackhacktutu:)happy 1676.0',
 'digitalhappynewyeardigital:)wagon railwaytutupgkhappy:)newyear 276.0',
 'railwaytutuwagonhappy:)tutu 20222022tutupgk:)hack 2827.0',
 'tutudigital2022:):)railway newyearnewyeartutu:):)2022 51.0',
 'pgkwagonrailwayrailway:)railway pgkwagonhackpgk:)happy 76.0',
 'wagonpgk

In [128]:
l_map = dict(zip(l, range(len(l))))
l_map

{'wagonwagonwagontutu:)wagon wagondigitalnewyearrailway:)happy 930.0': 0,
 'railwaypgkhappyrailway:)railway digitalhappynewyeardigital:)wagon 16.0': 1,
 'railwaytutupgkpgk:)digital pgkdigitalrailway:):)happy 879.0': 2,
 'tuturailwaydigital2022:)digital newyearnewyear20222022:)2022 8569.0': 3,
 'railwaytutupgkpgk:)digital pgkhappyrailwayhack:)railway 656.0': 4,
 'hacknewyearrailway:)pgk hackwagonhappy:)newyear 1021.0': 5,
 'happynewyearpgk:)railway hackwagon:):)tutu 4480.0': 6,
 'tutuwagon2022hack:)pgk newyearwagondigitalnewyear:)tutu 209.0': 7,
 'railwaytutupgkhappy:)newyear digitalhappynewyeardigital:)wagon 1680.0': 8,
 'newyearrailwaypgknewyear:)hack 2022wagondigital:)digital 3832.0': 9,
 'happy:)pgkrailway:)2022 wagonhackhacktutu:)happy 1676.0': 10,
 'digitalhappynewyeardigital:)wagon railwaytutupgkhappy:)newyear 276.0': 11,
 'railwaytutuwagonhappy:)tutu 20222022tutupgk:)hack 2827.0': 12,
 'tutudigital2022:):)railway newyearnewyeartutu:):)2022 51.0': 13,
 'pgkwagonrailwayrailway:)ra

In [129]:
df.combo = df.combo.map(l_map)

In [113]:
df.tail()

,st_code_snd,st_code_rsv,distance,y,predict,combo
4219365,tutunewyearwagonhack:)happy,newyearwagondigitaltutu:)pgk,1177.0,112.233333,None,1273617
4219366,happy:)pgkrailway:)2022,railwaypgkrailwaydigital:)2022,2078.0,115.816667,None,1170
4219367,wagonwagon2022newyear:)railway,newyearwagondigital2022:)railway,5336.0,190.718889,None,152216
4219368,newyearwagonhappytutu:)2022,newyearwagonpgkhack:)newyear,1886.0,49.583333,None,72919
4219369,pgkwagonpgk2022:)digital,pgkwagondigital2022:)digital,217.0,37.716667,None,199286


In [132]:
df = df.sort_values('combo')


In [133]:
res = {}

In [140]:
%%time
for combo in tqdm(range(len(l))):
    #
    # if row['predict'] is not None:
    #     continue
    t = df[df['combo'] == combo]

    res[combo] = t['y'].mean()

100%|██████████| 1273618/1273618 [56:08<00:00, 378.15it/s] 

CPU times: user 48min 3s, sys: 6min 12s, total: 54min 15s
Wall time: 56min 8s


In [141]:
def set_predict(combo):
    if combo in res:
        return res[combo]
    else:
        return None

df['predict'] = df['combo'].apply(set_predict)

In [142]:
df.loc[df['predict'] > 0]

,st_code_snd,st_code_rsv,distance,y,predict,combo
0,wagonwagonwagontutu:)wagon,wagondigitalnewyearrailway:)happy,930.0,185.200000,172.109059,0
2043340,wagonwagonwagontutu:)wagon,wagondigitalnewyearrailway:)happy,930.0,104.583333,172.109059,0
3856348,wagonwagonwagontutu:)wagon,wagondigitalnewyearrailway:)happy,930.0,121.683333,172.109059,0
3812616,wagonwagonwagontutu:)wagon,wagondigitalnewyearrailway:)happy,930.0,215.100000,172.109059,0
514419,wagonwagonwagontutu:)wagon,wagondigitalnewyearrailway:)happy,930.0,185.200000,172.109059,0
...,...,...,...,...,...,...
4219338,newyearrailway:)pgk:)digital,newyearwagondigitalhack:)happy,978.0,36.266667,36.266667,1273613
4219341,2022tutunewyeardigital:):),newyearwagonhackdigital:)wagon,35.0,29.600000,29.600000,1273614
4219346,railwaypgkrailwaypgk:)railway,newyearwagondigitalnewyear:)tutu,3795.0,254.250000,254.250000,1273615
4219351,tutudigitalwagonnewyear:)hack,tutupgkdigitalwagon:)railway,1388.0,75.766667,75.766667,1273616


In [161]:
df_filter = df[df['predict'] > 0]
print(df_filter.shape)

mean_squared_error(df_filter.y, df_filter.predict, squared = False)

(4139691, 6)


63.19228789561333

In [162]:
df_test = pd.read_csv('data/test.csv')

In [163]:
def join_row(row):
    return f"{row['st_code_snd']} {row['st_code_rsv']} {row['distance']}"

def join_row2(row):
    return f"{row['st_code_rsv']} {row['st_code_snd']} {row['distance']}"

df_test['combo'] = df_test.apply(join_row, axis=1)
df_test['combo2'] = df_test.apply(join_row2, axis=1)

In [164]:
df_test.combo2 = df_test.combo2.map(l_map)
df_test.combo = df_test.combo.map(l_map)

In [177]:
df_test['predict'] = None

In [178]:
def set_predict(row):
    combo = row['combo']
    combo2 = row['combo2']

    if combo in res:
        return res[combo]
    else:
        if combo2 in res:
            return res[combo2]
        else: return None

In [179]:
df_test['predict'] = df_test.apply(set_predict, axis=1)

In [180]:
df_test.loc[df_test['predict'] > 0]

,st_code_snd,st_code_rsv,date_depart_year,date_depart_month,date_depart_week,date_depart_day,date_depart_hour,fr_id,route_type,is_load,...,distance,snd_org_id,rsv_org_id,snd_roadid,rsv_roadid,snd_dp_id,rsv_dp_id,combo,combo2,predict
0,happy:)pgkwagon:):),pgk2022newyear:)pgk,2022,9,35,1,9,2261.0,3.0,0,...,2930.0,2974,2847,28,28,134,125,442574.0,NaN,189.088889
1,2022newyear:)newyear:):),hacktutu:)newyear:)2022,2022,8,34,22,14,1653.0,4.0,1,...,1728.0,2957,6340,18,18,102,16,28706.0,NaN,136.935949
2,happyrailwaytutu:)pgk,newyeartutu:)hack:)newyear,2022,5,21,26,16,643.0,3.0,1,...,4393.0,1664,2128,0,0,1,124,3402.0,12193.0,200.605254
4,happyrailwaytutu:)pgk,pgk2022newyear:)pgk,2022,5,20,17,6,2261.0,3.0,0,...,19.0,2974,1098,29,29,117,117,124797.0,NaN,5.104929
5,newyeardigitaldigitalhappy:)2022,tutupgkdigitalwagon:)railway,2022,10,41,11,22,100010.0,4.0,1,...,1766.0,1573,12504,21,21,110,125,15851.0,NaN,150.948808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182895,happy:)pgkwagon:):),wagon:):)hack:)pgk,2022,7,29,21,3,2342.0,3.0,1,...,984.0,1356,8896,32,32,29,68,22913.0,NaN,81.913321
1182898,pgknewyeardigitalhack:)hack,hack:)tutu:)railway,2022,6,26,27,19,539.0,4.0,1,...,1627.0,1416,14179,32,32,123,3,394593.0,NaN,192.851667
1182899,tutupgkdigitalrailway:)hack,happyrailwaytutu:)pgk,2022,9,39,27,16,1677.0,3.0,1,...,5096.0,2129,1664,27,27,125,1,1811.0,8777.0,282.518256
1182900,digitalhappynewyeardigital:)wagon,railwaytutupgkhappy:)newyear,2022,7,30,27,23,618.0,1.0,1,...,276.0,1316,2082,9,9,67,67,11.0,52.0,6.864732


In [181]:
df_test.shape

(1182903, 23)

In [182]:
df_test.loc[df_test['predict'] > 0].shape[0] /df_test.shape[0]

0.6290262177033958